**Binary Classification Of Sonar
Returns with an extra layer of neural network and standardization using pipeline**

In [26]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [27]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [28]:
# load dataset using pandas
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
X=dataset[:,0:60].astype(float)
Y=dataset[:,60]

In [29]:
# encode class values as integers - output values are letters 
# M or R - conver them to 0 or 1
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [30]:
print (Y[0:5])

['R' 'R' 'R' 'R' 'R']


**We are going to use
scikit-learn to evaluate the model using *stratified k-fold cross validation. *This is a resampling
technique that will provide an estimate of the performance of the model. **

**To use Keras models
with scikit-learn, we must use the KerasClassifier wrapper. This class takes a function that
creates and returns our neural network model. **

In [31]:
# create baseline model
def create_baseline():
    model = Sequential()
    model.add(Dense(60,input_dim=60,init='normal',activation='relu'))
    # Additional network to improve accuracy 
    # The idea here is that the network is given the opportunity to model all input variables
    # before being bottlenecked and forced to halve the representational capacity (if we made input 30 instead of 60)
    model.add(Dense(30, init= 'normal' , activation= 'relu' ))
    model.add(Dense(1,init='normal',activation='sigmoid'))
    
    #compile model
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

**Standardization.
    This is where the data is rescaled such that the mean value for each attribute is 0 and the standard deviation is 1. 
    This preserves Gaussian and Gaussian-like distributions whilst normalizing the central tendencies for each attribute
   Ex: StandardScaler **

** Pipeline
    Rather than performing the standardization on the entire dataset, it is
    good practice to train the standardization procedure on the training data within the pass of a
    cross validation run and to use the trained standardization instance to prepare the unseen test
    fold. **

** The pipeline is a wrapper that executes one or more models within a pass of the cross validation procedure. **

In [32]:
# evaluate model with standardized dataset
# estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=0) -- this code is used 
#when StandardScaler and Pipeline class for standarization are not applied

estimators=[]
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline,nb_epoch=100,batch_size=5,verbose=0)))

pipeline=Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 84.09% (4.39%)
